# DSC305 Final Project: Guess Your Doodling

In recent years, convolutional neural network(CNN) has literally dominates the realm of image classification. It outperforms previously used method like logistic regression and support vector machine.  ... ... . 
In this project, we will explore the power of CNN using the quickdraw dataset provided by Google. Quickdraw is an web application. It prompts user with a word, the user will draw an image for the word and the computer will guess the word will the user is drawing the image. The quickdraw dataset not only provided the final picture of users' hand-schatched images and the corresponding class labels, but also the sequence of strokes that the image is drawn by. Analyzing sequence of drawing would require recurrent neural network which is specialized for sequence data. In our project, we mainly focused on using CNN taking pixels as input to predict the class label of the hand-scatched image. Our final project consists of 2 parts. In part one, we would build a CNN that is able to classify hand-drawn images from 10 classes using Keras, a highe-level API for tensorflow.In part 2, we will deploy our model into a web application that mimic Quickdraw provided by Google.

Here we import all packages used in building our CNN model.

In [18]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.python.keras.backend as K
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import os
import gc
import cv2
import json
import shutil
import pickle

Before we start building our CNN model, let's prepare our dataset. The quickdraw dataset are provided in multiple formats, for exmaple '.npy' and '.json'. In this project, we choose to download '.npy' files and convert them to '.png' image files that can be used directly when training our CNN model. First we pull all '.npy' files using the last commands in the block below. Before we pull files, we also want to craete the directory to store those files. That's what first four lines are doing. Notice that there is an exclamation point at the beginning of each line. This indicates that those are shell commands, not Python code. Here we assume we are at a Linux environment and have *gsutil* installed.

In [8]:
!rm -rf data
!mkdir data
!mkdir data/npy_data
!mkdir data/png_data
!gsutil -q -m cp 'gs://quickdraw_dataset/full/numpy_bitmap/*.npy' data/npy_data

Once we have download our dataset, we can convert them into actual image files. As we are converting images, we will also split them into train, test and validation sets. So in our png_data directory, we create three subdirectories called "train", "test", "val". "Test" and "val"

In [19]:
file_dir = 'data/npy_data'
save_dir = 'data/png_data'
shutil.rmtree(save_dir)
os.mkdir(save_dir)
os.mkdir(save_dir + '/' + 'train')
os.mkdir(save_dir + '/' + 'test')
os.mkdir(save_dir + '/' + 'val')
files = os.listdir(file_dir)
X_data = []
y_data = []
TRAIN_SIZE = 5000
TEST_SIZE = 2000
VAL_SIZE = 1000
for index, file in enumerate(files):
    if index > 10:
        break
    train_images = np.load(file_dir+'/' + file)[:TRAIN_SIZE].copy()
    test_images = np.load(file_dir+'/' + file)[TRAIN_SIZE:TRAIN_SIZE + TEST_SIZE].copy()
    val_images = np.load(file_dir+'/' + file)[TRAIN_SIZE + TEST_SIZE : TRAIN_SIZE + TEST_SIZE + VAL_SIZE].copy()
    # Here I force the GC to collect unused memory 
    # to ensure there is enought memory to load other pictures
    gc.collect() 
    sub_dir = save_dir +'/'+'train'+ '/' + '_'.join(file.rstrip().split())
    os.mkdir(sub_dir)
    images = train_images.reshape(-1, 28, 28,1)
    for index, image in enumerate(images):
        cv2.imwrite(sub_dir + '/' + str(index)+'.png', image)
    
    sub_dir = save_dir +'/'+'test'+ '/' + '_'.join(file.rstrip().split())
    os.mkdir(sub_dir)
    images = test_images.reshape(-1, 28, 28,1)
    for index, image in enumerate(images):
        cv2.imwrite(sub_dir + '/' + str(index)+'.png', image)
    sub_dir = save_dir +'/'+'val'+ '/' + '_'.join(file.rstrip().split())
    os.mkdir(sub_dir)
    images = val_images.reshape(-1, 28, 28,1)
    for index, image in enumerate(images):
        cv2.imwrite(sub_dir + '/' + str(index)+'.png', image)
    gc.collect()
    

In [20]:
train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True
)
    

# test_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
TARGET_SIZE = (28,28)
BATCH_SIZE = 1024
CLASS_MODE = 'categorical'
ROOT_DIR = 'data/png_data'
COLOR_MODE = 'grayscale'
train_generator = train_datagen.flow_from_directory(
        ROOT_DIR + '/train',
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=CLASS_MODE,
        color_mode=COLOR_MODE)

validation_generator = val_datagen.flow_from_directory(
        ROOT_DIR + '/val',
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=CLASS_MODE,
        color_mode=COLOR_MODE)
test_generator = test_datagen.flow_from_directory(
        ROOT_DIR + '/test',
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=CLASS_MODE,
        color_mode=COLOR_MODE
)

Found 55000 images belonging to 11 classes.
Found 11000 images belonging to 11 classes.
Found 22000 images belonging to 11 classes.


In [21]:
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
save_obj(train_generator.class_indices, 'eleven_class_map')

In [22]:
K.clear_session()

model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(7 * 7 * 128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(11, activation='softmax')])
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [23]:
history= model.fit_generator(train_generator, epochs=50,
                                      validation_data=validation_generator)

Epoch 1/50
54/54 [==============================] - 8s 148ms/step - loss: 18.6460 - acc: 0.4809 - val_loss: 0.7751 - val_acc: 0.7655
Epoch 2/50
54/54 [==============================] - 7s 121ms/step - loss: 0.7157 - acc: 0.7798 - val_loss: 0.5313 - val_acc: 0.8419
Epoch 3/50
54/54 [==============================] - 7s 130ms/step - loss: 0.5639 - acc: 0.8291 - val_loss: 0.4759 - val_acc: 0.8575
Epoch 4/50
54/54 [==============================] - 7s 129ms/step - loss: 0.4970 - acc: 0.8494 - val_loss: 0.4331 - val_acc: 0.8738
Epoch 5/50
54/54 [==============================] - 7s 128ms/step - loss: 0.4507 - acc: 0.8625 - val_loss: 0.4283 - val_acc: 0.8746
Epoch 6/50
54/54 [==============================] - 7s 130ms/step - loss: 0.4175 - acc: 0.8726 - val_loss: 0.4179 - val_acc: 0.8749
Epoch 7/50
54/54 [==============================] - 7s 127ms/step - loss: 0.3921 - acc: 0.8809 - val_loss: 0.4034 - val_acc: 0.8825
Epoch 8/50
54/54 [==============================] - 7s 127ms/step - loss: 0

In [24]:
loss, acc = model.evaluate_generator(test_generator, workers=16)

In [25]:
print('loss:', loss)
print('acc:', acc)

loss: 0.45300136371092364
acc: 0.9013636


In [ ]:
model.save('eleven_class.h5')